# **Soni - do**
# **Generating Music with Machine Learning**


#### Author: Sonia Cobo
#### Date: July 2021

### Music is associated with emotions, experiences and creativity, all o them considered human's qualities. 

### Though this project doesn't have a hypothesis per se it was done to prove that technology has advanced so much that a machine, that cannot experience these feelings, can generate music.

### ....

In [12]:
# data augmentation - dividir canciones, modificarlas para tener mas datos

# Data

### Music files have been imported to Python in MIDI format. MIDI (Musical Instrument Digital Interface) is a technical standard that describes a communications protocol, digital interface, and electrical connectors that connect a wide variety of electronic musical instruments and computers. They don't contain actual audio data and are small in size. They explain what notes are played, when they're played, and how long or loud each note should be.

### To simplify the project only MIDI files consisting of a single instrument were chosen. In this case, the chosen instrument is piano and the type of songs is classical. 

### These songs have been obtained from the following datasets: http://www.piano-midi.de/ and https://www.mfiles.co.uk/classical-midi.htm
### As guidance for the project the following example was considered: 'How to Generate Music using a LSTM Neural Network in Keras' by Sigurður Skúli.


In [13]:
# no descargardas aun: https://github.com/Skuldur/Classical-Piano-Composer/tree/master/midi_songs
# https://drive.google.com/file/d/1qnQVK17DNVkU19MgVA4Vg88zRDvwCRXw/view

## Import all libraries

In [1]:
# data manipulation
import numpy as np
import pandas as pd 
from sqlalchemy import create_engine

# manipulate midi files
import glob
from music21 import *
#from music21 import converter, instrument, note, chord, meter, stream, duration, corpus
import pygame

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# route files
import os
import sys

# ml model
import pickle

import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Flatten

# my libraries
import utils.mining_data_tb as md
from utils.folders_tb import read_json
#import utils.visualization_tb as vis
from utils.sql_tb import MySQL


pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Paths

In [2]:
# The route of this file is added to the sys path to be able to import/export functions
sep = os.sep
def route (steps):
    """
    This function appends the route of the file to the sys path
    to be able to import files from/to other foders within the EDA project folder.
    """
    route = os.getcwd()
    for i in range(steps):
        route = os.path.dirname(route)
    sys.path.append(route)
    return route

In [3]:
# path to raw data
path = route(1) + sep + "data" + sep + "raw_data" + sep
# path to data in the right key
path_1 = route(1) + sep + "data" + sep + "converted_data" + sep
# path to compiled notes list
path_2 = route(1) + sep + "data" + sep + "notes" + sep
# path to generated models
path_3 = route(1) + sep + "models" + sep
# path to generated midi files
path_4 = route(1) + sep + "reports" + sep

## Midi file exploration

### The Python library 'Music21' has been used to read and manipulate MIDI files. This library has the necessary classes to allow us to read music. 

### Hablar de frecuencia y la transpuesta de fourier----------------------------------

### To start working with MIDI files these need to be converted to Score objects, which are a subclass for handling multi-part music. Once this is done it is possible to use the library build-in classes to view the file information.

In [4]:
# All information from the midi file (i.e. notes, pitch, chord, time signature, etc) is contained within the component list

def info_midi (path, filename):
    """
    It returns all midi file information given its path and filename

    """
    # Convert to Score object
    file = converter.parse(path + filename)
    components = []
    # read file information
    for element in file.recurse():  
        components.append(element)
    return components

components = info_midi(path, "alb_esp1.mid")
#components

### Usig pieces from the same key, in this case C major / A minor assists the model to not go off key.

In [ ]:
# Ya hecho
# md.transpose_key(path, path_1)

In [ ]:
# mejorar graficos mirar video visualizacion borja 
# separar barritas, nombres horizontales

### Knowing all the file components it is possible to select the useful ones for our prediction. The main components to generate new melodies are notes, rests and chords.

### Note objects contain information about the pitch, octave, and offset of the note. Pitch refers to the frequency of the sound, or how high or low it is and is represented with the letters [A, B, C, D, E, F, G] or [Do, Re, Mi, Fa, Sol, La, Si] in Spanish. Octave refers to which set of pitches you use on a piano. Offset refers to where the note is located in the piece.

(añadir visualizacion de lo de arriba para enseñar que es pitch, etc)

### Rests are the silences in the piece.
### Chord objects are a set of notes that are played at the same time.


In [ ]:
#  probar que pasa si le meto cualquier midi, de varios instrumentos, etc
# data augmentation = min_notes_freq

## EDA

### Relevant information from midi file is encoded and saved into an array.

### We append the pitch of every note object using its string notation since the most significant parts of the note can be recreated using the string notation of the pitch. And we append every chord by encoding the id of every note in the chord together into a single string, with each note being separated by a dot. 

In [5]:
# Each midi file contains notes and chords. These two properties will be the input and output of the LSTM network so 
# they need to be taken out from all midi files. 

def get_notes_per_song(path, filename, save_path, save_name):
    """
    This function extracts all the notes, rests and chords from one midi file
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file, filename (str)
    """
    components = info_midi(path, filename)
    note_list = []
    
    for element in components:
        # note pitches are extracted
        if isinstance(element, note.Note):
            note_list.append(str(element.pitch))
        # chords are extracted
        elif isinstance(element, chord.Chord):
            note_list.append(".".join(str(n) for n in element.normalOrder))    
        # rests are extracted
        elif isinstance(element, note.Rest):
            note_list.append("NULL")    #further transformation needs this value as str rather than np.nan

    # save list with all componenets extracted
    with open(save_path + save_name, "wb") as filepath:
        pickle.dump(note_list, filepath)
    
    return note_list

In [6]:
one_song = get_notes_per_song(path_1, "C_alb_esp1.mid", path_2, "one_notes")

In [7]:
def get_all_notes(path, save_name, save_path):
    """
    This function extracts all the notes, rests and chords from all midi files 
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file     
    """
    all_notes = []
    list_path = os.listdir(path)
    for filename in list_path:
        output = get_notes_per_song(path, filename, save_path, save_name)
        all_notes += output
        
    return all_notes

In [8]:
all_notes = get_all_notes(path = path_1, save_path = path_2, save_name = "all_notes")

In [9]:
print(len(all_notes))
print(len(set(all_notes)))

696938
604


In [10]:
# Load notes and chords previously separated
def load_notes (path, filename):
    """
    Load the note list containing pitches, rests and chords.
    
    Param: Path of the saved note list, and its name as string
    """
    with open(path + filename, "rb") as f:
        loaded_notes = pickle.load(f)
        return loaded_notes

In [11]:
load_chopin = load_notes(path_2, "notes_chopin")


### The model will be first trained with a small proportioned of the songs to expedite time. Once the model is tunned properly all songs will be passed to improve its training.

### Now that all notes, rests and chords are in a list, these will be transformed from categorical data to integer-based numerical data. It is necessary to create input sequences for the network and their respective outputs. The output for each input sequence will be the first note or chord that comes after the sequence of notes in the input sequence in our list of notes.

### The length of each sequence will be 100 notes/chords for now. This means that to predict the next note in the sequence the network has the previous 100 notes to help make the prediction

In [12]:
# generate dataframe

def create_dataframe(path, save_path, save_name):
    """
    Create a dataframe and a list with all pieces title and the notes, rests and chrods included in the piece. 

    Param: Path of the midi file, filename (str), path where the list will be saved and its name.
    """
    list_path = os.listdir(path)
    piece_list = []
    notes = []
    for elem in list_path:
        output = get_notes_per_song(path, elem, save_path, save_name)
        piece_list.append(elem[:-4])
        notes.append(output)

    df = pd.DataFrame.from_dict({"Piece":piece_list, "Notes":notes}, orient="index")
    df = df.transpose()

    notes = [item for sublist in notes for item in sublist]

    return df, notes

In [13]:
df_music, note_list = create_dataframe(path= path_1, save_path = path_2, save_name = "all_notes")

In [21]:
n = [item for sublist in note_list for item in sublist]
len(set(n))


604

In [ ]:
df_music

,Piece,Notes
0,C_alb_esp1,"[NULL, E5, B5, A5, A5, G5, A5, B5, C6, D6, B5,..."
1,C_alb_esp2,"[NULL, G3, 0.4.7, NULL, G3, 0.4.7, 7.0, E4, F4..."
2,C_alb_esp3,"[E5, NULL, G#5, NULL, B5, NULL, A5, A5, F5, NU..."
3,C_alb_esp4,"[A5, NULL, E5, NULL, F5, NULL, A5, NULL, G#5, ..."
4,C_alb_esp5,"[NULL, 0.4, 11.2, 2.5, 11.2, 0.4, 9.0, 5.9, 4...."
...,...,...
337,C_waldstein_1,"[NULL, 0.4, NULL, 0.4, NULL, 0.4, NULL, 0.4, N..."
338,C_waldstein_2,"[NULL, C3, NULL, A3, NULL, A3, A3, NULL, 4.8, ..."
339,C_waldstein_3,"[NULL, A5, G5, A5, G5, A5, G5, A5, G5, A5, G5,..."
340,C_waltz-op18-grande-brillante,"[D5, D5, NULL, D5, NULL, D5, D5, NULL, D5, NUL..."


In [265]:
# Save cleaned data in the appropriate folder, in this case it is the folder 'data'
def save_dataframe(dataframe, dataframe_name, steps):
    """
    Create a cvs file from a dataframe.
    Param: Dataframe to save, dataframe name as string to save the data with that name
    """
    dataframe.reset_index(inplace = True)
    if "index" in dataframe.columns:
        dataframe.drop(columns = "index", inplace = True)
    dataframe.to_csv(route(steps) + os.sep + "data" + os.sep + "output" + os.sep + dataframe_name + ".csv", index=False)
    return "Your file has been saved"

In [266]:
df_music = save_dataframe(df_music, "df_music", 1)

In [267]:
df_music

'Your file has been saved'

## Pre-processing

In [176]:
def prepare_sequences(notes, min_note_occurence, sequence_length, step):
    """ 
    This function creates the input and output sequences used by the neural network.
    It returns the x and y of the model.

    Param: 
        Note: List containing all notes, rests and chords
        Sequence_length: Lenght of notes given to the model to help predict the next
        Step: Step (int) between one input sequence and the next one
    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # Calculate occurence
    note_freq = {}
    for elem in notes:
        note_freq[elem] = note_freq.get(elem, 0) + 1

    ignored_notes = set()
    for k, v in note_freq.items():
        if note_freq[k] < min_note_occurence:
            ignored_notes.add(k)
    
    
    print('Unique words before ignoring:', len(pitchnames))
    print('Ignoring words with occurence <', min_note_occurence)
    pitchnames = sorted(set(pitchnames) - ignored_notes)
    print('Unique words after ignoring:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, step): 
        # remove ignored notes from the note list   
        if len(set(notes[i: i+ sequence_length + 1]).intersection(ignored_notes)) == 0:
            network_input.append(notes[i:i + sequence_length])
            network_output.append(notes[i + sequence_length])
    # array of zeros
    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), len(pitchnames)))
    # exchange note values for their integer-code
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
        y[i, note_to_int[network_output[i]]] = 1

    return x, y

In [177]:
x, y = prepare_sequences(notes=load_chopin, min_note_occurence=1, sequence_length=100, step=3)  # length y step pueden variar  

# o coger menos canciones - añadirlo en la memoria


Total unique notes: 109
Unique words before ignoring: 109
Ignoring words with occurence < 1
Unique words after ignoring: 109


In [107]:
print(x.shape)
print(y.shape)

(474, 100, 109)
(474, 109)


## Visualization

## Model creation

There are four different types of layers:

LSTM layers is a Recurrent Neural Net layer that takes a sequence as an input and can return either sequences (return_sequences=True) or a matrix.

Dropout layers are a regularisation technique that consists of setting a fraction of input units to 0 at each update during the training to prevent overfitting. The fraction is determined by the parameter used with the layer.

Dense layers or fully connected layers is a fully connected neural network layer where each input node is connected to each output node.

The Activation layer determines what activation function our neural network will use to calculate the output of a node.

In [159]:
# upgraded lstm model
def create_network(num_units, num_dense, input_shape):
    """
    Builds and compiles a simple RNN model
    Param:
              num_units: Number of units of a the simple RNN layer
              num_dense: Number of neurons in the dense layer followed by the RNN layer
              input_shape: input_shape
    """
    model = Sequential()
    model.add(Bidirectional(LSTM(num_units, input_shape=input_shape, return_sequences=True)))
    model.add(Dense(num_dense))
    model.add(Dropout(0.3))
    model.add(LSTM(num_units, return_sequences=True))
    model.add(Dense(num_dense))
    model.add(Dropout(0.3))
    model.add(LSTM(num_units))
    model.add(Dense(num_dense))
    model.add(Dropout(0.3))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

    return model

In [160]:
model = create_network(num_units=x.shape[0], num_dense=x.shape[2], input_shape=(x.shape[1],x.shape[2]))


In [140]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 100, 474)          1107264   
_________________________________________________________________
dense_21 (Dense)             (None, 100, 109)          51775     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100, 109)          0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 100, 474)          1107264   
_________________________________________________________________
dense_22 (Dense)             (None, 100, 109)          51775     
_________________________________________________________________
dropout_18 (Dropout)         (None, 100, 109)          0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 474)             

In [141]:
model.fit(x, y, epochs=20)#, batch_size=128)

Epoch 1/20
15/15 [==============================] - 25s 1s/step - loss: 5.1869
Epoch 2/20
15/15 [==============================] - 18s 1s/step - loss: 4.2229
Epoch 3/20
15/15 [==============================] - 17s 1s/step - loss: 4.0632
Epoch 4/20
15/15 [==============================] - 17s 1s/step - loss: 4.0789
Epoch 5/20
15/15 [==============================] - 17s 1s/step - loss: 4.1008
Epoch 6/20
15/15 [==============================] - 17s 1s/step - loss: 4.1042
Epoch 7/20
15/15 [==============================] - 17s 1s/step - loss: 4.0860
Epoch 8/20
15/15 [==============================] - 18s 1s/step - loss: 4.0748
Epoch 9/20
15/15 [==============================] - 18s 1s/step - loss: 4.0216
Epoch 10/20
15/15 [==============================] - 17s 1s/step - loss: 4.0077
Epoch 11/20
15/15 [==============================] - 17s 1s/step - loss: 3.8819
Epoch 12/20
15/15 [==============================] - 17s 1s/step - loss: 3.8662
Epoch 13/20
15/15 [==============================

In [142]:
# save the model
model.save(path_3 + "model_2_lstm.h5")

In [143]:
# load the model 
# Baseline LSTM model - model_1_lstm
# Updated LSTM model - model_2_lstm

model_2_lstm = tf.keras.models.load_model(path_3 + "model_2_lstm.h5")
model_2_lstm.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 100, 474)          1107264   
_________________________________________________________________
dense_21 (Dense)             (None, 100, 109)          51775     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100, 109)          0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 100, 474)          1107264   
_________________________________________________________________
dense_22 (Dense)             (None, 100, 109)          51775     
_________________________________________________________________
dropout_18 (Dropout)         (None, 100, 109)          0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 474)             

In [144]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [169]:
def generate_notes(notes, model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(notes)-100-1)

    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = notes[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music
    for note_index in range(100):
        prediction_input = np.zeros((1, 100, len(pitchnames)))
        for j, note in enumerate(pattern):
            prediction_input[0, j, note_to_int[note]] = 1.0
        preds = model.predict(prediction_input, verbose=0)[0] 
        next_index = sample(preds, temperature=temperature)
        next_note = int_to_note[next_index]

        pattern = pattern[1:]
        pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns

In [171]:
prediction_output, patterns = generate_notes(notes=load_chopin, model=model_2_lstm, temperature=1.0)
print(prediction_output)

['E6', 'B5', 'F5', 'E2', 'NULL', '4.8', '1.6', '9', 'NULL', 'NULL', '4.7.11', '5.10', 'NULL', 'NULL', 'E4', 'E2', 'D6', 'F2', 'A5', 'C2', '9.0.4', 'B2', '0.4', 'G4', '9.0.4', '2.5', 'E2', 'C2', '2.8', 'B1', 'E-6', '9.0.3', 'A2', '2.4', 'C#6', 'NULL', '8.11.2.4', 'NULL', '0.4', '6.9.0', 'NULL', '11.4', 'C#6', '2.5', 'NULL', '0.4', '0.4', '2.4', '5.11', '6.9.0', '0.4', '5.10', 'C4', 'NULL', 'NULL', 'D2', 'C6', 'C#4', '5.7', '9.2', '5.10', 'C2', '1.5', '3.9', '11.4', 'NULL', '2.5', 'C2', '2.5', 'A2', 'B2', 'NULL', 'NULL', 'B4', 'D2', 'NULL', '5.11', '6.9.11', '0.3.6', 'NULL', 'E2', '5.7', 'A2', 'D4', '9.0.4', '5.7', '10.2', 'NULL', '9.1', 'E3', 'D4', 'E2', '2.8', '10.2', 'C4', '0.4', 'B1', 'E2', 'A1', 'G4']


# Output

In [172]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_1_lstm1.mid")   # first output 01/07/2021

    return midi_stream

In [173]:
create_midi = create_midi(prediction_output, patterns, path_4)


In [174]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [175]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_1_lstm1.mid")

In [20]:
import gc
gc.collect()

16353